# NATURAL LANGUAGE PROCESSING

### Exercice 1 : ANALYSE DE REPRÉSENTATION DES MOTS AVEC BOW, TD-IDF, Word2Vc,GloVe

##### I- BAG OF WORDS:


a- Vocabulaire :

Nous voulons avoir une liste de mots uniques à partir du document:

In [29]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Documents
documents = [
    "Le chat mange la souris.",
    "La souris mange du fromage.",
    "Le chien chasse le chat.",
    "Le chien et le chat sont amis."
]
vectorizer = CountVectorizer()   # Initialize the CountVectorizer
X = vectorizer.fit_transform(documents) # Fit and transform the documents
vocabulary = vectorizer.get_feature_names_out() # Get the vocabulary (feature names)
vocabulary # print out vocabulary

array(['amis', 'chasse', 'chat', 'chien', 'du', 'et', 'fromage', 'la',
       'le', 'mange', 'sont', 'souris'], dtype=object)

b-MATRICE DE TERMES

on veut l'occurence de chaque mot dans chaque document sous forme de matrice

In [30]:
# Convert the term-document matrix to a DataFrame for better readability
import pandas as pd
term_document_matrix = pd.DataFrame(X.toarray(), columns=vocabulary)
term_document_matrix


,amis,chasse,chat,chien,du,et,fromage,la,le,mange,sont,souris
0,0,0,1,0,0,0,0,1,1,1,0,1
1,0,0,0,0,1,0,1,1,0,1,0,1
2,0,1,1,1,0,0,0,0,2,0,0,0
3,1,0,1,1,0,1,0,0,2,0,1,0


On a une matrice de termes convertie en data frame qui represente nos mots uniques ainsi notre bag of words.

##### II- TD-Idf


a-TF

Nous voulons avoir la fréquence de chaque terme dans chaque document

In [31]:
# Calculer la fréquence des termes (TF)
tf = term_document_matrix.div(term_document_matrix.sum(axis=1), axis=0)
tf

,amis,chasse,chat,chien,du,et,fromage,la,le,mange,sont,souris
0,0.000000,0.0,0.200000,0.000000,0.0,0.000000,0.0,0.2,0.200000,0.2,0.000000,0.2
1,0.000000,0.0,0.000000,0.000000,0.2,0.000000,0.2,0.2,0.000000,0.2,0.000000,0.2
2,0.000000,0.2,0.200000,0.200000,0.0,0.000000,0.0,0.0,0.400000,0.0,0.000000,0.0
3,0.142857,0.0,0.142857,0.142857,0.0,0.142857,0.0,0.0,0.285714,0.0,0.142857,0.0


b- IDF

Nous voulons avoir l'inverse du nombre de documents contenant un terme sur le nombre total de documents

In [32]:
import numpy as np
# Calculer IDF pour chaque terme
N = len(documents)  # Nombre total de documents
df = (term_document_matrix > 0).sum(axis=0)  # Nombre de documents contenant le terme
idf = np.log(N / (df))  # Calculer l'IDF

# Convertir l'IDF en DataFrame pour une meilleure lisibilité
idf_df = pd.DataFrame(idf, index=vocabulary, columns=["IDF"])
print("\nInverse Document Frequency (IDF) :")
idf_df


Inverse Document Frequency (IDF) :


,IDF
amis,1.386294
chasse,1.386294
chat,0.287682
chien,0.693147
du,1.386294
et,1.386294
fromage,1.386294
la,0.693147
le,0.287682
mange,0.693147


c- la matrice TF-IDF

Ici, il suffit de multiplier la fréquence de chaque terme dans chaque document (TF) par l'inverse des documents contenant le terme sur le nombre total des documents (IDF)

In [27]:
# Calculer la matrice TF-IDF
tf_idf = tf * idf
tf_idf

,amis,chasse,chat,chien,du,et,fromage,la,le,mange,sont,souris
0,0.000000,0.000000,0.057536,0.000000,0.000000,0.000000,0.000000,0.138629,0.057536,0.138629,0.000000,0.138629
1,0.000000,0.000000,0.000000,0.000000,0.277259,0.000000,0.277259,0.138629,0.000000,0.138629,0.000000,0.138629
2,0.000000,0.277259,0.057536,0.138629,0.000000,0.000000,0.000000,0.000000,0.115073,0.000000,0.000000,0.000000
3,0.198042,0.000000,0.041097,0.099021,0.000000,0.198042,0.000000,0.000000,0.082195,0.000000,0.198042,0.000000


In [34]:
# Convertir TF-IDF en DataFrame pour une meilleure lisibilité
tf_idf_df = pd.DataFrame(tf_idf, columns=vocabulary)

print("\nTF-IDF :")
print(tf_idf_df)


TF-IDF :
       amis    chasse      chat     chien        du        et   fromage  \
0  0.000000  0.000000  0.057536  0.000000  0.000000  0.000000  0.000000   
1  0.000000  0.000000  0.000000  0.000000  0.277259  0.000000  0.277259   
2  0.000000  0.277259  0.057536  0.138629  0.000000  0.000000  0.000000   
3  0.198042  0.000000  0.041097  0.099021  0.000000  0.198042  0.000000   

         la        le     mange      sont    souris  
0  0.138629  0.057536  0.138629  0.000000  0.138629  
1  0.138629  0.000000  0.138629  0.000000  0.138629  
2  0.000000  0.115073  0.000000  0.000000  0.000000  
3  0.000000  0.082195  0.000000  0.198042  0.000000  


##### III- Word2Vec

a.CONCEPTS

Quand on veut transformer nos données << textes >> en données "numériques", et on veut que ces données numériques aient un "sens", on 
utilise du "word embeddings"; ainsi les "mots similaires" auront des "données numériques similaires". Un outil du word embedding est le word2vec.
Ainsi, pour inclure < du context > à nos mots, on utilise le Word2Vec. 

Word2Vec est une méthode pour représenter les mots sous forme de vecteurs numériques qui capturent les similartés sémantiques et syntatiques entre les mots.

Pour ce faire, on a deux stratégies que Word2Vec utilise pour inclure du context: 


01-CBOW(Continuous Bag Of Word): pour augmenter le contexte, CBOW va utiliser les mots environnants(les mots voisins) pour prédire le mot au milieu.  Plus rapide à entraîner que Skip-gram, surtout avec des mots fréquents et Performe mieux avec des ensembles de données volumineux où les relations entre les mots sont bien établies.



02-Skip-Gram : Pour augmenter le contexte, Skip-gram va utiliser le mot au milieu pour prédire les mots environnants; il peut mieux représenter les mots rares et capturer des relations syntaxiques plus précises, mais il est plus lent à entraîner. il peut capturer des relations plus fines et des analogies syntaxiques, ce qui en fait un choix préféré pour certaines tâches spécifiques comme l'analyse syntaxique.

In [22]:
from gensim.models import Word2Vec
import re